In [1]:
from src.eval.agents.dockerized_agent import DockerizedAgent
from src.docker.docker_session_config import DockerSessionConfig

In [2]:
config = DockerSessionConfig(volumes={
    '/Users/Maria.Tigina/PycharmProjects/agents-eval-data/docker': {'bind': '/mnt/vol', 'mode': 'rw'},
    '/Users/Maria.Tigina/PycharmProjects/agents-eval-data/docker_cp': {'bind': '/mnt/vol1', 'mode': 'cp'}
})
agent = DockerizedAgent(config)

In [3]:
with agent.docker_session() as session_interface:
    out = session_interface.execute_command("touch /tmp/test.txt")
    print(f"Created file /tmp/test.txt")
    
    out = session_interface.execute_command('ls -la /tmp')
    print(f"List of files in /tmp:\n{out}")
    
    out = session_interface.execute_command("touch /mnt/vol/test_mnt_vol.txt")
    print(f"Created file /mnt/vol/test_mnt_vol.txt")
    
    out = session_interface.execute_command('ls -la /mnt/vol')
    print(f"List of files in /mnt/vol:\n{out}")
    
    session_interface.recreate()
    out = session_interface.execute_command('ls -la /tmp')
    print(f"Recreated container; list of files in /tmp:\n{out}")

Created file /tmp/test.txt
List of files in /tmp:
total 8
drwxrwxrwt 1 root root 4096 Mar  6 11:37 .
drwxr-xr-x 1 root root 4096 Mar  6 11:37 ..
drwxr-xr-x 3 root root   96 Mar  6 11:28 _mnt_vol1
-rw-r--r-- 1 root root    0 Mar  6 11:37 test.txt
Created file /mnt/vol/test_mnt_vol.txt
List of files in /mnt/vol:
total 4
drwxr-xr-x 3 root root   96 Mar  6 11:37 .
drwxr-xr-x 1 root root 4096 Mar  6 11:37 ..
-rw-r--r-- 1 root root    0 Mar  6 11:37 test_mnt_vol.txt
Recreated container; list of files in /tmp:
total 8
drwxrwxrwt 1 root root 4096 Mar  6 11:37 .
drwxr-xr-x 1 root root 4096 Mar  6 11:37 ..
drwxr-xr-x 3 root root   96 Mar  6 11:28 _mnt_vol1


In [ ]:
from src.utils.print_utils import tee, ColorCodes
import time

test_project_path = '/Users/Maria.Tigina/PycharmProjects/agents-eval-data/docker/emptyProject'
print(f"Project path: {test_project_path}")

question = "List all available git branches and their last commit message"
prompt = tee("Create gradle project", color=ColorCodes.OKGREEN)

config = DockerSessionConfig(
    image="ghcr.io/jetbrains-research/ideformer-plugin/simulator:latest",
    command=["/project"],
    ports={8080: 8080},
    working_dir="/project",
    volumes={
            test_project_path: {"bind": "/project", "mode": "cp"}
    },
    interactive=True,
    interactive_interpreter="bash",
)
agent = DockerizedAgent(config)

max_steps = 20
executed_commands = [None] * max_steps

with agent.docker_session() as session_interface:
    for i in range(0, max_steps):
        # sleep to avoid throttling
        time.sleep(1)
        # Tho brock should be first for every step
        prompt += tee(f"\nTho {i}: ##", color=ColorCodes.OKGREEN)
    
        thought = complete(str(prompt), stop_at=["##", "\nAct", "\nTho", "\nObs"])
        prompt += tee(thought, color=ColorCodes.OKBLUE) + tee("##", color=ColorCodes.OKGREEN)
        
        # Act block is the next
        prompt += tee(f"\nAct {i}: ", color=ColorCodes.OKGREEN)
        action = complete(str(prompt), is_in=["COMMAND", "COMMUNICATE", "RESET", "FINISH"])
        prompt += tee(action, color=ColorCodes.OKBLUE) + tee(" ##", color=ColorCodes.OKGREEN)

        if action == "RESET":
            subject = complete(str(prompt), is_in=[str(j) for j in range(0, i)])
        else:
            subject = complete(str(prompt), stop_at=["##", "\nAct", "\nTho", "\nObs"])
        prompt += tee(subject, color=ColorCodes.OKBLUE) + tee("##", color=ColorCodes.OKGREEN)
        
        if action == "COMMAND":
            result = session_interface.execute_command(subject).output
            executed_commands[i] = subject
            # add an indicator if result is empty
            if len(result.split("\n")) == (1 if is_interactive else 0):
                result += "\n<NO_OUTPUT>"

        elif action == "COMMUNICATE":
            result = input(subject)
            if len(result) == 0:
                result = "<NO_INPUT>"

        elif action == "RESET":
            session_interface.recreate()
            # reapply all the commands up to the target step
            target = int(subject)
            j = 0
            try:
                for j in range(0, target):
                        if executed_commands[j] is not None:
                            session_interface.execute_command(executed_commands[j])

                result = f"Successfully rerun commands up to step {subject}."
            except Exception as e:
                result = f"Failed to reset to step {subject}: {e}. Last executed step: {j}."
                break
            
            # remove all the executed commands after the target step
            executed_commands[target:] = [None] * (max_steps - target)

        elif action == "FINISH":
            correct = input(f"Model response '{subject}'. "
                            "Print 'stop' to finish or reply to the model")
            if correct.lower() == "stop":
                break
            else:
                result = correct

        prompt += tee(f"\nObs {i}: ##", color=ColorCodes.OKGREEN) + tee(result, color=ColorCodes.OKCYAN) + tee("##", color=ColorCodes.OKGREEN)